In [2]:
import pandas as pd
import requests
import zipfile
import tarfile
from io import BytesIO

## Reading corpus data

In [ ]:
# how to read data file using pandas from github repo
url= 'https://github.com/scrosseye/CLEAR-Corpus/blob/main/CLEAR_corpus_final.xlsx?raw=True'
myfile = requests.get(url)

df=pd.read_excel(url, sheet_name='Data', engine='openpyxl')
print(df.info())


## Corpus of Sentences rated with Human Complexity Judgments

In [ ]:
file_path = './../data/complexity_ds_en.csv'
df = pd.read_csv(file_path)
print(df.info())

## REALEC Corpus

In [21]:
# open tar.gz file name Exam2016.tar.gz in data folder and print all files in that that have .txt and .json extension, then store them in a list
# all content of txt file as string and from json file read the content of key 'ielts', 'text' and store them as pandas dataframe

file_path = './../data/Exam2016.tar.gz'
count = 0
dict_text = {}
dict_json = {}
data = []
with tarfile.open(file_path, "r:gz") as tar:
    count += 1
    for tarinfo in tar:
        if tarinfo.name.endswith(".txt") or tarinfo.name.endswith(".json"):
            file_name = tarinfo.name
            file_name = file_name.split('/')[1].split('.')[0]
            with tar.extractfile(tarinfo) as f:
                if tarinfo.name.endswith(".txt"):
                    str_sentence = f.read()
                    dict_text[file_name] = str_sentence
                else:
                    json_data = pd.read_json(f, orient='index').T
                    dict_json[file_name] = json_data


for key in dict_text.keys():
    datum = []
    sentence = dict_text[key]
    json_data = dict_json[key]
    datum = [sentence, json_data['ielts'][0], json_data['CEFR_level'][0], json_data['work_type'][0], json_data['year'][0]]
    data.append(datum)

df_pd = pd.DataFrame(data, columns=['sentence', 'ielts', 'CEFR_level', 'work_type', 'year'])
df_pd = df_pd[df_pd['CEFR_level'] != '']
df_pd.to_csv('./../data/Exam2016.csv', index=False, header=True)
            

## MMLL - EFCAMDAT

In [ ]:
efcamdat_file_path = './../data/Final database (main prompts).xlsx'
df = pd.read_excel(efcamdat_file_path, sheet_name='Sheet 1')
efcamdat_df = df[['writing_id', 'cefr', 'cefr_numeric', 'level', 'grade', 'wordcount', 'mtld', 'text', 'text_corrected']]

In [ ]:
efcamdat_file_path = './../data/Final database (alternative prompts).xlsx'
df = pd.read_excel(efcamdat_file_path, sheet_name='Sheet 1')
efcamdat_df = df[['writing_id', 'cefr', 'cefr_numeric', 'level', 'grade', 'wordcount', 'mtld', 'text', 'text_corrected']]

## NEWSELA Corpus

In [ ]:
url = "https://s3.amazonaws.com/newsela-research-corpora/newsela_article_corpus_with_scripts_2016-01-29.1.zip"
# read zip file from URL and also read the given csv file in the zip file
response = requests.get(url)
df = pd.DataFrame()
with zipfile.ZipFile(BytesIO(response.content)) as z:
    with z.open('newsela_article_corpus_2016-01-29/articles_metadata.csv') as f:
        df = pd.read_csv(f)
        print(df.info())
    df['content'] = df['filename'].apply(lambda x: z.open('newsela_article_corpus_2016-01-29/articles/' + x).read())


## ASSET Dataset

In [8]:
username = "facebookresearch"
repo_name = "asset"
folder_path = "dataset"
url = f"https://api.github.com/repos/{username}/{repo_name}/contents/{folder_path}"
response = requests.get(url)
response.raise_for_status()
files = response.json()
file_series = []
for file in files:
    if file['type'] == 'file' and 'valid' in file['name']:
            print(file['name'])
            file_contents = []
            content_url = file['download_url']
            content_response = requests.get(content_url)
            content_response.raise_for_status()
            content = content_response.text
            file_contents.extend(content.splitlines())
            series = pd.Series(file_contents)
            file_series.append(series)

df = pd.concat(file_series, axis=1)
df.columns =['Original', 'Simplified1', 'Simplified2', 'Simplified3', 'Simplified4', 'Simplified5', 'Simplified6', 'Simplified7',
'Simplified8', 'Simplified9', 'Simplified10']

asset.valid.orig
asset.valid.simp.0
asset.valid.simp.1
asset.valid.simp.2
asset.valid.simp.3
asset.valid.simp.4
asset.valid.simp.5
asset.valid.simp.6
asset.valid.simp.7
asset.valid.simp.8
asset.valid.simp.9
